<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ติดตั้งไลบรารีสำหรับ Google Sheet, Notion และ env
!pip install gspread oauth2client notion-client python-dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

# โหลดค่าจาก .env ที่อยู่ใน Drive
env_path = '/content/drive/MyDrive/Automation_Keys/Notion_Expense/.env'
load_dotenv(dotenv_path=env_path)

# ดึงค่าตัวแปร
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")
SHEET_ID = os.getenv("SHEET_ID")

In [ ]:
# อัปโหลดไฟล์ service_account.json เข้าสู่ Colab memory
from google.colab import files
files.upload()  # เลือกไฟล์จาก Drive หรือ External HDD ก็ได้

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
client = gspread.authorize(creds)

# เปิด Sheet ด้วย SHEET_ID จาก .env (ไม่ต้อง hardcode แล้ว)
sheet = client.open_by_key(SHEET_ID).sheet1

In [ ]:
def parse_message(text):
    try:
        parts = text.strip().split()
        category = parts[0]
        amount = float(parts[1])
        return category, amount
    except:
        return None, None

In [ ]:
from notion_client import Client

notion = Client(auth=NOTION_TOKEN)

def push_to_notion(category, amount):
    notion.pages.create(
        parent={"database_id": NOTION_DB_ID},
        properties={
            "Categories": {"select": {"name": category}},
            "Amount": {"number": amount}
        }
    )

In [ ]:
def sync_sheet_to_notion():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue

        category, amount = parse_message(row['Message'])
        if not category or not amount:
            print(f"❌ Format ผิดที่ row {idx+2}")
            continue

        try:
            push_to_notion(category, amount)
            sheet.update_cell(idx+2, 4, "YES")
            print(f"✅ Synced row {idx+2}: {category} {amount}")
        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")

# เรียกฟังก์ชัน
sync_sheet_to_notion()

In [ ]:
import datetime

# ฟังก์ชันนี้ใช้บันทึกข้อความจากผู้ใช้ LINE ลงใน Google Sheet
# โดยจะเก็บเวลา (timestamp), user ID, ข้อความ, และสถานะการ sync
def log_message(user_id, message):
    now = datetime.datetime.now().isoformat()  # เวลาปัจจุบันในรูปแบบ ISO (เช่น 2025-06-17T15:00:00)
    log_sheet.append_row([now, user_id, message, "NO"])  # เพิ่มแถวใหม่ลงใน Sheet โดยยังไม่ mark ว่า sync แล้ว

In [ ]:
# ฟังก์ชันนี้ใช้ตรวจสอบว่า ข้อความมีรูปแบบ "[หมวดหมู่] [จำนวนเงิน]" หรือไม่
# เช่น "food 150", "กาแฟ 65" → จะ return category และจำนวนเงิน
# ถ้า format ไม่ถูกต้อง จะ return (None, None)

def is_expense_message(text):
    try:
        parts = text.strip().split()  # ตัดช่องว่างหน้า/หลัง และแยกคำออกเป็น list
        category = parts[0]           # คำแรกคือหมวดหมู่ เช่น "food"
        amount = float(parts[1])      # คำที่สองต้องแปลงเป็นจำนวนเงินได้ เช่น 150
        return category, amount
    except:
        return None, None  # ถ้าผิดพลาดใด ๆ (เช่นพิมพ์ผิด) จะคืนค่า None ทั้งคู่